In [6]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
from ultralytics import YOLO
from concurrent.futures import ThreadPoolExecutor

In [7]:
model = YOLO("yolov8n.pt")

In [8]:
# Function to detect persons in a batch of images
def detect_persons(images):
    results = model(images, verbose=False)
    batch_boxes = []
    for result in results:
        boxes = np.array([box.xyxy.numpy() for box in result.boxes if box.cls.numpy()[0] == 0]).reshape(-1, 4)
        batch_boxes.append(boxes)
    return batch_boxes

# Function to create a single bounding box that encapsulates all detected persons
def get_combined_bounding_box(boxes):
    if len(boxes) == 0:
        return None
    x_min = np.min(boxes[:, 0])
    y_min = np.min(boxes[:, 1])
    x_max = np.max(boxes[:, 2])
    y_max = np.max(boxes[:, 3])
    return (x_min, y_min, x_max, y_max)

# Function to crop persons from an image
def crop_persons(image, combined_box):
    if combined_box is None:
        return None  # Return None if no person detected
    x1, y1, x2, y2 = combined_box
    crop = image.crop((x1, y1, x2, y2))
    return crop

# Function to process a batch of images
def process_batch(image_paths, output_dir, progress_bar):
    images = [Image.open(image_path) for image_path in image_paths]
    images_np = [np.array(image) for image in images]
    batch_boxes = detect_persons(images_np)
    
    for image_path, image, boxes in zip(image_paths, images, batch_boxes):
        combined_box = get_combined_bounding_box(boxes)
        crop = crop_persons(image, combined_box)
        if crop is not None:
            output_path = os.path.join(output_dir, os.path.basename(image_path))
            crop.save(output_path)
        progress_bar.update(1)

# Main pipeline function
def person_cropping_pipeline(input_dir, output_dir, batch_size=4):
    os.makedirs(output_dir, exist_ok=True)
    image_paths = [os.path.join(input_dir, fname) for fname in os.listdir(input_dir) if fname.endswith(('.png', '.jpg', '.jpeg'))]

    with tqdm(total=len(image_paths)) as progress_bar:
        with ThreadPoolExecutor() as executor:
            for i in range(0, len(image_paths), batch_size):
                batch = image_paths[i:i + batch_size]
                executor.submit(process_batch, batch, output_dir, progress_bar)

In [ ]:
input_dir = './images'
output_dir = './cropped_images'
person_cropping_pipeline(input_dir, output_dir)

  0%|          | 57/411677 [00:11<36:03:12,  3.17it/s]